In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [22]:
# 读取数据
train = pd.read_csv('data/first_round_training_data.csv')
test = pd.read_csv('data/first_round_testing_data.csv')
train.head(10)
train.describe()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10
count,6000.000000,6.000000e+03,6000.000000,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6000.000000,6.000000e+03,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03
mean,485.198954,1.952379e+03,406.348529,189.205842,5.868163e+00,5.598597e+00,272.300821,22.133231,5.823664e+04,35.902066,5.351032e+04,8.842895e+04,202.732522,6.271156e+03,1.285985e+06,3.400877e+03,3.166863e+04,4.545394e+05,7.860735e+05,1.989520e+07
std,11401.971393,5.419588e+04,12018.898941,2621.779049,8.631865e+00,7.686898e+00,2027.460514,302.172768,3.173068e+06,101.629348,1.894866e+06,2.854543e+06,2754.503943,2.585508e+05,9.850723e+07,1.999508e+05,1.116889e+06,2.847995e+07,5.885757e+07,1.529785e+09
min,0.000003,1.683797e-07,0.000005,0.000002,9.985942e-07,9.810610e-07,0.000624,0.023699,3.962831e-04,0.001024,6.457597e-10,5.493777e-09,0.000001,4.344229e-06,1.635864e-06,5.456653e-05,4.411611e-03,3.011145e-03,2.366726e-03,1.974795e-03
25%,0.088221,4.765771e-02,0.102304,0.118314,7.416296e-01,3.818154e-01,0.152059,0.035407,5.930812e-01,0.101478,3.100601e-02,3.173294e-02,0.098178,1.319373e-01,1.150362e-01,1.443224e-01,1.135895e-01,1.115708e-01,1.267717e-01,1.253268e-01
50%,1.050304,9.405962e-01,1.044908,1.068467,2.225808e+00,2.684398e+00,0.600827,2.931083,5.930812e-01,0.727611,9.449563e-01,1.030499e+00,0.945947,8.297855e-01,7.539155e-01,9.864325e-01,5.891701e-01,5.922906e-01,6.644951e-01,6.705953e-01
75%,10.938565,1.569199e+01,10.665397,9.902064,6.680183e+00,7.117767e+00,2.374035,17.850021,6.783967e+00,13.969790,2.784766e+01,3.001000e+01,9.803828,6.052952e+00,5.736029e+00,7.422284e+00,4.637157e+00,4.773050e+00,5.571728e+00,4.905501e+00
max,713682.207601,3.034568e+06,864530.632179,139767.494732,8.367195e+01,4.117541e+01,35698.591553,20086.397753,1.738083e+08,517.197610,1.197060e+08,1.615907e+08,128491.084741,1.572225e+07,7.630284e+09,1.488592e+07,7.789923e+07,2.187522e+09,4.558485e+09,1.184961e+11


In [23]:
# 特征列表
features = ["Parameter1","Parameter2","Parameter3","Parameter4","Parameter5","Parameter6","Parameter7","Parameter8","Parameter9","Parameter10"]

In [24]:
# 标签转化
def quality_encoder(x):
    return {'Excellent':0,'Good':1,'Pass':2,'Fail':3}[x]

train['label'] = train.Quality_label.apply(quality_encoder)

# 哑变量
train['label_Excellent'] = 1*(train['label'] == 0)
train['label_Good'] = 1*(train['label'] == 1)
train['label_Pass'] = 1*(train['label'] == 2)
train['label_Fail'] = 1*(train['label'] == 3)
train.head(10)

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,...,Attribute7,Attribute8,Attribute9,Attribute10,Quality_label,label,label_Excellent,label_Good,label_Pass,label_Fail
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,...,177.243120,338.729256,2.021704,0.079526,Pass,2,0,0,1,0
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,3.723330,15.376190,0.986973,4.634376,Fail,3,0,0,0,1
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,...,0.064196,0.576302,33.875790,1.813727,Fail,3,0,0,0,1
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,0.029777,0.246726,27.117165,0.081819,Fail,3,0,0,0,1
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,0.055053,0.014725,13.569707,18.138496,Fail,3,0,0,0,1
5,0.004062,14.556483,0.786945,0.010545,0.000525,0.001623,2286.523413,0.035407,0.593081,1.010385,...,0.576380,123.057489,16.133884,0.598517,Good,1,0,1,0,0
6,0.438449,1.232559,2.882699,0.610757,1.600654,0.464037,0.600827,17.850021,0.051850,0.010192,...,0.035493,0.146690,41.285376,5.985572,Good,1,0,1,0,0
7,48159.917401,0.002987,14.863813,0.063287,1.434060,0.314162,0.600827,17.850021,0.051850,0.010192,...,0.615609,3.321156,0.013034,0.045039,Good,1,0,1,0,0
8,1.456601,52.381578,20.625283,366.074831,1.434060,0.258497,0.600827,17.850021,0.051850,0.010192,...,0.029777,0.007033,3.397996,20.322889,Pass,2,0,0,1,0
9,0.000109,4.784654,31.916672,20.672388,1.284806,0.175007,0.600827,17.850021,0.051850,0.010192,...,1.356619,1.081445,0.056060,0.413597,Good,1,0,1,0,0


In [25]:
# 本地多分类准确率评估
local_train = train.iloc[:5000,:].copy()
local_valid = train.iloc[5000:,:].copy()

# GradientBoostingClassifier为GBDT的分类类,max_depth决策树最大深度，数据少特征少用3，否则10~100;
#learning_rate权重缩减系数，即步长，0~1，默认1；n_estimators弱学习器的最大迭代次数，与learning_rate配合调参
#  random_state: if int,is the seed used by the random number generator
model = GradientBoostingClassifier(max_depth=3,learning_rate=0.1,n_estimators=100,random_state=666)
model.fit(local_train.loc[:,features],local_train.label)

local_valid['prediction'] = model.predict(local_valid.loc[:,features])
(local_valid.label == local_valid.prediction).mean()

0.463

In [28]:
# 线上提交
model = GradientBoostingClassifier(max_depth=3,learning_rate=0.1,n_estimators=100,random_state=666)
model.fit(train.loc[:,features],train.label)

test['prediction'] = model.predict(test.loc[:,features])
test['prob_Excellent'] = 0.0
test['prob_Good'] = 0.0
test['prob_Pass'] = 0.0
test['prob_Fail'] = 0.0
test.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] = model.predict_proba(test.loc[:,features])
test.head(10)

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,prediction,prob_Excellent,prob_Good,prob_Pass,prob_Fail
0,0,0.235279,0.195680,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852,0,0.449316,0.263670,0.213523,0.073491
1,0,0.043799,0.523976,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257,3,0.036857,0.180220,0.096383,0.686540
2,0,21.466737,0.523976,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257,3,0.092080,0.187559,0.091526,0.628835
3,0,0.703795,0.010192,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850,1,0.080136,0.509901,0.172738,0.237225
4,0,10.447960,0.010192,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850,1,0.097155,0.514454,0.161785,0.226607
5,0,733.439920,0.010192,99.698641,3.202776,31.368042,0.477823,0.464037,0.038483,17.850021,0.051850,1,0.091479,0.512079,0.155303,0.241138
6,0,0.149962,0.010192,2.910066,2.187656,331.744593,0.533331,0.464037,0.038483,17.850021,0.051850,1,0.080319,0.567216,0.159941,0.192524
7,0,0.060635,0.010192,0.016492,0.007373,7183.436876,0.533331,0.464037,0.038483,17.850021,0.051850,1,0.340208,0.435838,0.105398,0.118556
8,0,971.284623,0.010192,29.954312,3.639672,0.136383,0.477823,0.381815,0.038483,17.850021,0.051850,1,0.124574,0.441646,0.173236,0.260544
9,0,1372.664549,0.010192,2.103451,0.531944,5.802723,0.477823,0.464037,0.038483,17.850021,0.051850,1,0.115976,0.534260,0.149876,0.199888


In [30]:
# 提交用
prediction = test.groupby(['Group'],as_index=False)['prob_Excellent','prob_Good','prob_Pass','prob_Fail'].mean()
prediction.columns = ['Group','Excellent ratio','Good ratio','Pass ratio','Fail ratio']
prediction.to_csv('baseline.csv',index=False)